In [96]:
import pandas as pd
import numpy as np
import math

In [97]:
articles_df = pd.read_csv('shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']

In [98]:
articles_df.head(1)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en


In [99]:
interactions_df = pd.read_csv('users_interactions.csv')

In [100]:
interactions_df.head(1)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN


In [101]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [102]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [103]:
interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

In [104]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]

In [105]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [106]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].max()
)
        
interactions_full_df = interactions_full_df.reset_index()

In [107]:
interactions_full_df.head(2)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
1,-1007001694607905623,-6623581327558800021,1.0,1487240080


In [108]:
from sklearn.model_selection import train_test_split

split_ts = 1475519545
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()


In [109]:
#iterations_by_all_users = pd.pivot_table(interactions_train_df, index='personId',columns='contentId', values='eventStrength',aggfunc='sum' )

In [110]:
#iterations_by_all_users=iterations_by_all_users.fillna(0)


In [111]:
#iterations_by_all_users.head(2)

Для начала необходимо построить матрицу, в которой по столбцам будут находиться id статей, по строкам — id пользователей, а на пересечениях строк и столбцов — оценка взаимодействия пользователя со статьёй. Если взаимодействия не было, в соответствующей ячейке должен стоять ноль.

Задание 6.1

Найдите оценку взаимодействия пользователя с ID -1032019229384696495 со статьёй с ID 943818026930898372. Результат округлите до двух знаков после точки-разделителя.

In [121]:
ratings = pd.pivot_table(
    interactions_train_df,
    values="eventStrength",
    index="personId",
    columns="contentId",
).fillna(0)
round(ratings.loc["-1032019229384696495", "943818026930898372"], 2)

2.32

In [112]:
#iterations_by_all_users.loc['-1032019229384696495']['943818026930898372']

In [113]:
#n_users = iterations_by_all_users.shape[0]

In [114]:
#iterations_by_all_users_df = iterations_by_all_users.copy()

Задание 6.2

Найдите среднее арифметическое всех чисел в получившемся массиве. Результат округлите до трёх знаков после точки-разделителя.

In [122]:
ratings_m = ratings.values #Замечательный способ массив значений из ДатаФрейма забрать
ratings_m.mean()

0.016668620737604063

In [115]:
#iterations_by_all_users = np.array(iterations_by_all_users)
#iterations_by_all_users.mean()

Задание 6.3

Постройте матрицу схожести. Для этого вычислите все попарные коэффициенты корреляции для матрицы, полученной в предыдущем задании. Для каждой пары учитывайте только ненулевые значения (так как нулевые обозначают отсутствие взаимодействия и не интересуют нас). Выведите результат, полученный в ячейке с третьим индексом по строкам и сороковым — по столбцам.

In [133]:
similarity_users = np.zeros((len(ratings_m), len(ratings_m)))
for i in (range(len(ratings_m)-1)):
    for j in range(i+1, len(ratings_m)):
     
        mask_uv = (ratings_m[i] != 0) & (ratings_m[j] != 0)
        ratings_v = ratings_m[i, mask_uv]
        ratings_u = ratings_m[j, mask_uv]

        similarity_users[i,j] = np.corrcoef(ratings_v, ratings_u)[0, 1] #np.corrcoef дает матрицу корреляций двух векторов. Нам надо любой значение с побочной диагонали
        similarity_users[j,i] = similarity_users[i,j]

similarity_users[3,40]

-0.3333333333333333

In [116]:
#corr_matrix = np.zeros([n_users,n_users])
#for i in range(n_users):
#    for j in range(n_users):
#        v = iterations_by_all_users[i]
#        w = iterations_by_all_users[j]
#        id_vect = v*w
#        if id_vect.mean() != 0:
#            v1=list()    
#            w1=list()
#            for t,x in enumerate(id_vect):
#                if x!=0:
#                    v1.append(v[t])
#                    w1.append(w[t])
#            
#            v1 = np.array(v1)
#            w1 = np.array(w1)
#            v1 = v1 - v1.mean()
#            w1 = w1 - w1.mean()
#            v1 = v1/np.linalg.norm(v1)
#            w1 = w1/np.linalg.norm(w1)
#            corr_matrix[i][j] = v1@w1    
#        else:
#            corr_matrix[i][j] = 0


In [117]:
#corr_matrix[3][40]

Теперь у нас есть матрицы схожести пользователей. Их можно использовать для построения рекомендаций. Чтобы это сделать, надо реализовать следующий алгоритм.

Для каждого пользователя:

* Найти пользователей с похожестью больше 0.
* Для каждой статьи вычислить долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.
* Порекомендовать статьи (не более 10) с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).

Задание 6.4
Постройте рекомендательную систему по алгоритму, описанному выше. Найдите первую рекомендацию для строки 35 (если считать с нуля).

In [141]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
) #Заполним по пользователю списки номеров старей тренирововчной выборки

interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
) #Заполним по пользователю списки номеров старей тестовой выборки

interactions['true_test'] = [ [] if x is np.NaN else x for x in interactions['true_test'] ]
#Уберем Nan из тестовых данных

prediction_user_based = []
for i in range(len(similarity_users)):
    users_sim = similarity_users[i] > 0 #Маска по пользователям,  с которыми у данного пользователя > 0 корреляция
    if not any(users_sim): #Если везде ноль, то добавляем пустой список в качестве рекомендации
        prediction_user_based.append([])
    else:
        tmp_recommend = np.argsort(ratings_m[users_sim].sum(axis=0))[::-1]
        #выбираем из таллицы с рейтингами по инексам users_sim т.е. инексам мест, где стоят положительные корреляции с нашим юзером,
        #строки. И по каждой строке сумму берем. Чем больше сумма - тем больше популярность, поэтом сортируем эти индексы и берем по убыванию через [::-1]
        tmp_recommend = ratings.columns[tmp_recommend]
        recommend = np.array(tmp_recommend)[~np.in1d(tmp_recommend, interactions.iloc[i]["true_train"])][:10]
        prediction_user_based.append(list(recommend))
interactions['prediction_user_based'] = prediction_user_based
prediction_user_based[35][0]

'-5148591903395022444'

In [118]:
#corr_matrix

In [119]:
#corr_matrix_df = pd.DataFrame(corr_matrix, index=iterations_by_all_users_df.index, columns=iterations_by_all_users_df.index)
#corr_matrix_df

In [120]:
#for user in iterations_by_all_users_df.index:

#user = '-1007001694607905623'

#intersect_users = iterations_by_all_users_df.T[iterations_by_all_users_df.T[user]>0]
#intersect_users 


Задание 6.5
Вычислите точность полученного предсказания. Ответ округлите до трёх знаков после точки-разделителя.

In [161]:
def calc_precision(column):
    return ( interactions.apply(  lambda row:len(set(row['true_test']).intersection(
                set(row[column]))) /min(len(row['true_test']) + 0.001, 10.0), axis=1)).mean()

In [162]:
round(calc_precision('prediction_user_based'), 3)

0.005

Задание 6.6
Теперь реализуем рекомендательную систему с использованием SVD.

Разложите матрицу взаимодействий пользователей со статьями с помощью функции svd из модуля scipy. Найдите максимальное значение в получившейся матрице U. Результат округлите до двух знаков после точки-разделителя.

In [165]:
from scipy.linalg import svd

In [166]:
U, sigma, V = svd(ratings)
U.max()

0.7071067811865497

Значения матрицы с сингулярными числами отсортированы по убыванию. Допустим, мы хотим оставить только первые 100 компонент и получить скрытые представления размерности 100. Для этого необходимо оставить 100 столбцов в матрице U, только первые 100 значений из sigma (и сделать из них диагональную матрицу) и 100 строк в матрице V. Затем необходимо перемножить преобразованные матрицы.

Задание 6.7

Найдите сумму всех элементов в новой сингулярной матрице. Ответ округлите до двух знаков после точки-разделителя.

In [167]:
k = 100
s = np.diag(sigma[:k])
U = U[:, 0:k]
V = V[0:k, :]

round(s.sum(), 2)

2096.43

Теперь мы можем сделать предсказание по полученной матрице.

Примечание. Помните, что не нужно учитывать статьи, которые уже были просмотрены пользователем.

Найдите для каждого пользователя статьи с наибольшими оценками в восстановленной матрице.

Задание 6.8

Вычислите качество полученного предсказания, используя всё ту же метрику точности. Ответ округлите до трёх знаков после точки-разделителя.

In [169]:
new_ratings = pd.DataFrame(
    U@s@V, index=ratings.index, columns=ratings.columns
)
top_k = 10
predictions = []

for personId in interactions.index:
    prediction = (
        new_ratings.loc[personId].sort_values(ascending=False).index.values
    )

    predictions.append(
        list(
            prediction[
                ~np.in1d(prediction, interactions.loc[personId, "true_train"])
            ]
        )[:top_k]
    )

interactions["prediction_svd"] = predictions

calc_precision("prediction_svd")

0.012023152289763838